In [7]:
import pandas as pd
import numpy as np
import requests, zipfile, io
import os

In [8]:
# global vars
pyear = "20205year"
missing_state_value = np.nan 
# file_prefix = 'ss16p'
file_prefix = 'psam_p04'
mass_dl_url = 'https://www2.census.gov/programs-surveys/acs/data/pums/2020/5-Year/csv_p'
dest = f"{pyear}/pums_states/"
parent_dir = "./"

In [9]:
# response = requests.get(mass_dl_url+'az'+'.zip')
# z = zipfile.ZipFile(io.BytesIO(response.content))
# z.extractall(f"{dest}az")

In [10]:
# pums = pd.read_csv(f"{parent_dir}{pyear}/pums_states/az/{file_prefix}az.csv")
#virinchi# pums = pd.read_csv(f"{dest}az/{file_prefix}az.csv")
pums = pd.read_csv(f"{dest}/az/{file_prefix}.csv")

C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\808698182.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  pums = pd.read_csv(f"{dest}/az/{file_prefix}.csv")


In [11]:
#virinchi cell#
pums = pums[ (pums['AGEP']>20) & ((pums['ESR'] == 1) | (pums['ESR'] == 2) | (pums['ESR'] == 4) | (pums['ESR'] == 5) )]
#virinchi cell#

In [12]:
pums

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
3,P,2016000000060,8,1,201,4,4,1086849,14,44,...,17,15,12,14,17,29,24,27,11,5
4,P,2016000000060,8,2,201,4,4,1086849,20,45,...,22,25,19,20,25,33,31,31,17,7
8,P,2016000000094,8,1,201,4,4,1086849,7,51,...,9,14,12,12,8,3,8,13,9,12
13,P,2016000000242,8,1,500,4,4,1086849,17,21,...,17,17,17,16,3,3,17,16,17,17
14,P,2016000000295,8,1,123,4,4,1086849,31,40,...,31,9,49,30,8,8,30,27,29,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331330,P,2020HU1135506,8,1,119,4,4,1006149,11,58,...,24,9,8,18,16,16,16,8,20,13
331331,P,2020HU1135506,8,2,119,4,4,1006149,7,58,...,16,7,5,13,15,12,13,6,14,13
331332,P,2020HU1135570,8,1,101,4,4,1006149,1,70,...,0,0,0,0,0,0,0,0,0,0
331334,P,2020HU1135612,8,1,121,4,4,1006149,10,62,...,10,19,15,11,10,8,20,5,14,3


In [13]:
pums["ftNAICSP"] = pums["NAICSP"].astype(str).apply(lambda x : x[:2])

In [14]:
industry_naics = pd.read_excel("AZ - Industry - NAICS2.xls", sheet_name=2, usecols=["NAICS", "Description"])
industry_naics.dropna(inplace=True)
industry_naics.NAICS = industry_naics.NAICS.astype(int).astype(str)

In [15]:
industry_naics = industry_naics.append({'NAICS':'92', 'Description':'Government and Military'}, ignore_index=True)

C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\3645387805.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  industry_naics = industry_naics.append({'NAICS':'92', 'Description':'Government and Military'}, ignore_index=True)


In [16]:
industry_naics

,NAICS,Description
0,11,"Agriculture, Forestry, Fishing and Hunting"
1,21,"Mining, Quarrying, and Oil and Gas Extraction"
2,22,Utilities
3,23,Construction
4,31,Manufacturing
5,42,Wholesale Trade
6,44,Retail Trade
7,48,Transportation and Warehousing
8,51,Information
9,52,Finance and Insurance


In [17]:
# Remove 3ms,4ms and 92
# pums = pums[pums["ftNAICSP"].isin(["3M", "4M"])==False]

pums.loc[pums["ftNAICSP"]=="32","ftNAICSP"] = "31" 
pums.loc[pums["ftNAICSP"]=="33","ftNAICSP"] = "31" 
pums.loc[pums["ftNAICSP"]=="4M","ftNAICSP"] = "31" # manufacturing

pums.loc[pums["ftNAICSP"]=="3M","ftNAICSP"] = "44"  # retail
pums.loc[pums["ftNAICSP"]=="45","ftNAICSP"] = "44"

pums.loc[pums["ftNAICSP"]=="49","ftNAICSP"] = "48" 

In [18]:
#virinchi# pums_titles = pd.read_csv(f"../Mappings/PUMS{pyear}_SOCPTitles.csv")
#virinchi# naics_titles = pd.read_csv(f"../Mappings/PUMS{pyear}_NAICSPTitles.csv")
#virinchi# pums_migsp_titles = pd.read_csv("../PUMS/PUMS_MIGSPNames.csv")
pums_titles = pd.read_csv(f"PUMS{pyear}_SOCPTitles.csv")
naics_titles = pd.read_csv(f"PUMS{pyear}_NAICSPTitles.csv")
pums_migsp_titles = pd.read_csv("PUMS_MIGSPNames.csv")

In [19]:
pums_migsp_titles

,MIGSP,Name,Short
0,1,Alabama,AL
1,2,Alaska,AK
2,4,Arizona,AZ
3,5,Arkansas,AR
4,6,California,CA
5,8,Colorado,CO
6,9,Connecticut,CT
7,10,Delaware,DE
8,11,District of Columbia,DC
9,12,Florida,FL


In [20]:
pums_migsp_titles = pums_migsp_titles.append({'MIGSP':99, 'Name':'All States Combined', 'Short':'ASC'}, ignore_index=True)

C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\61219139.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pums_migsp_titles = pums_migsp_titles.append({'MIGSP':99, 'Name':'All States Combined', 'Short':'ASC'}, ignore_index=True)


In [21]:
pums.shape

(133638, 289)

In [22]:
# 92 is govt adm + mil
# 90 is adm only

adm_titles = naics_titles[naics_titles.Title.str.startswith('Adm-')]

temp_df = pums.loc[pums.NAICSP.isin(adm_titles.NAICSP.values)]#NAICSP.apply(lambda x:x[:2]).values)]

if temp_df.size > 0:
    temp_df.loc[:,"ftNAICSP"] = '90'
    pums = pd.concat([pums, temp_df])
    pums = pums.reset_index(drop=True)

C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\2324683804.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.loc[:,"ftNAICSP"] = '90'


In [23]:
adm_titles

,NAICSP,Title
255,92113,Adm-Public Finance Activities
256,92119,Adm-Other General Government And Support
257,9211MP,Adm-Executive Offices And Legislative Bodies
258,923,Adm-Administration Of Human Resource Programs
266,928P,Adm-National Security And International Affairs
267,92M1,Adm-Administration Of Environmental Quality An...
268,92M2,Adm-Administration Of Economic Programs And Sp...
269,92MP,"Adm-Justice, Public Order, And Safety Activities"


In [24]:
temp_df

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80,ftNAICSP
4,P,2016000000060,8,2,201,4,4,1086849,20,45,...,25,19,20,25,33,31,31,17,7,90
91,P,2016000002003,8,2,100,4,4,1086849,16,53,...,16,5,5,15,16,16,5,27,27,90
115,P,2016000002592,8,1,700,4,4,1086849,26,27,...,24,7,50,40,11,29,43,38,11,90
146,P,2016000003333,8,1,107,4,4,1086849,13,47,...,14,3,23,24,12,14,4,13,13,90
306,P,2016000006121,8,1,115,4,4,1086849,26,29,...,9,41,26,25,7,45,26,25,47,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331107,P,2020HU1130289,8,2,105,4,4,1006149,7,39,...,8,14,10,8,8,7,14,3,11,90
331119,P,2020HU1130588,8,2,208,4,4,1006149,10,58,...,9,21,10,15,3,10,17,17,9,90
331240,P,2020HU1132666,8,1,900,4,4,1006149,38,39,...,39,60,38,70,43,56,31,13,14,90
331297,P,2020HU1134686,8,1,800,4,4,1006149,30,50,...,11,50,55,42,28,24,11,36,49,90


In [25]:
pums.shape

(140796, 289)

In [26]:
pums.ftNAICSP.unique()

array(['52', '92', '71', '61', '31', '62', '53', '54', '72', '44', '56',
       '48', '23', '22', '51', '42', '81', '11', '21', '55', '90'],
      dtype=object)

In [27]:
pums = pums[pums["ftNAICSP"]!="na"]

In [28]:
pums.ftNAICSP = pums.ftNAICSP.apply(str)

In [29]:
pums.MIGPUMA = pums.MIGPUMA.astype(float)
pums.MIGSP = pums.MIGSP.astype(float)
pums.PWGTP = pums.PWGTP.astype(float)

In [30]:
pums_outofaz = pums[(pums.MIGPUMA > 2) & (pums.MIGSP != 4) & (pums.MIGSP <= 56)]

pums_outofaz = pums_outofaz[pums_outofaz.NAICSP.notna()]
pums_outofaz = pums_outofaz[pums_outofaz.SOCP.notna()]

In [31]:
industry_naics

,NAICS,Description
0,11,"Agriculture, Forestry, Fishing and Hunting"
1,21,"Mining, Quarrying, and Oil and Gas Extraction"
2,22,Utilities
3,23,Construction
4,31,Manufacturing
5,42,Wholesale Trade
6,44,Retail Trade
7,48,Transportation and Warehousing
8,51,Information
9,52,Finance and Insurance


In [32]:
pums_titles.head()

,SOCP,Title
0,bbbbbb,(Less Than 16 Years Old/Nilf Who Last Worked M...
1,1110XX,Chief Executives And Legislators
2,111021,General And Operations Managers
3,112011,Advertising And Promotions Managers
4,112021,Marketing Managers


In [33]:
pums_migsp_titles.head()
state_id_to_name =  pums_migsp_titles.groupby("MIGSP")["Name"].unique()

In [34]:
total_weights_bynaics = pums.groupby("ftNAICSP")["PWGTP"].sum()
total_weights_bysocp = pums.groupby("SOCP")["PWGTP"].sum()

In [35]:
total_weights_bynaics['0'] = total_weights_bynaics.values.sum()
total_weights_bysocp['0'] = total_weights_bysocp.values.sum()

In [36]:
len(pums_outofaz.SOCP.unique())

432

# Part 1

In [37]:
grouped_by_naics_wgt = pums_outofaz.groupby(['ftNAICSP'])["PWGTP"].sum()
# grouped_by_naics_socp = pums_outofaz.groupby(['ftNAICSP'])['SOCP'].apply(list)
grouped_by_naicsp = pums_outofaz.groupby(['ftNAICSP'])['NAICSP'].apply(lambda x : list(np.unique(list(x))))

In [38]:
# p1 = []
# for i in industry_naics.index:
#     ind_name = industry_naics.iloc[i]["Description"]
#     ind_naics = industry_naics.iloc[i]["NAICS"]
#     if ind_naics in grouped_by_naics_wgt.keys():
#         row = [ind_naics, ind_name, grouped_by_naics_wgt[ind_naics], grouped_by_naicsp[ind_naics]]
#         p1.append(row)
#     else:
#         row = [ind_naics, ind_name, 0,[]]
#         p1.append(row)

In [39]:
# p1_out = pd.DataFrame(p1, columns=["NAICS", "Description", "MovedToAZLastYear", "NAICSPs"])

In [40]:
# p1_out.to_csv("21_Part1_MovedToAZ_0705.csv",index=False)

### Part 1 all

In [41]:
# p1_all = []
# for key in grouped_by_naics_wgt.keys():
#     ind_name = industry_naics.loc[industry_naics.NAICS==key]["Description"].values
#     ind_naics = industry_naics.loc[industry_naics.NAICS==key]["NAICS"].values
#     if len(ind_name) > 0:
#         ind_name = ind_name[0]
#         ind_naics = ind_naics[0]
#     else:
#         ind_name = ''
#         ind_naics = ''
#     row = [ind_naics, ind_name, grouped_by_naics_wgt[key], grouped_by_naicsp[key]]
#     p1_all.append(row)

In [42]:
# p1_all_out = pd.DataFrame(p1_all, columns=["NAICS", "Description", "MovedToAZLastYear", "NAICSPs"])

In [43]:
# p1_all_out.to_csv("pums_Part1_MovedToAZ_0705.csv",index=False)

### Part 1 All also by state

In [44]:
grouped_naicsp_state_wgt = pums_outofaz.groupby(['ftNAICSP', 'MIGSP'])["PWGTP"].sum()

# irrelevant
grouped_by_naicsp_state = pums_outofaz.groupby(['ftNAICSP', 'MIGSP'])['NAICSP'].apply(lambda x : list(np.unique(list(x))))

In [45]:
# all states combined sum 
all_states_combinedp1 = pums_outofaz.groupby(['ftNAICSP'])["PWGTP"].sum()

In [46]:
all_states_combinedp1

ftNAICSP
11      832.0
21      457.0
22      652.0
23     6212.0
31     7675.0
42     3183.0
44    12554.0
48     4441.0
51     2306.0
52     6915.0
53     2279.0
54    11571.0
55      224.0
56     7118.0
61     8341.0
62    15277.0
71     3177.0
72     7221.0
81     3944.0
90     5266.0
92     8279.0
Name: PWGTP, dtype: float64

In [47]:
p1_all_state = []


for key in grouped_by_naics_wgt.keys():
    ind_name = industry_naics.loc[industry_naics.NAICS==key]["Description"].values
    ind_naics = industry_naics.loc[industry_naics.NAICS==key]["NAICS"].values
    if len(ind_name) > 0:
        ind_name = ind_name[0]
        ind_naics = ind_naics[0]
    else:
        ind_name = ''
        ind_naics = ''
    for s in grouped_naicsp_state_wgt[key].keys():
        row = [ind_naics, ind_name, s, state_id_to_name[int(s)][0], grouped_naicsp_state_wgt[key][s], grouped_by_naicsp_state[key][s]]
        p1_all_state.append(row)
    for i in pums_migsp_titles.index[:-1]:
        migsp = pums_migsp_titles.iloc[i]["MIGSP"]
        if migsp not in grouped_naicsp_state_wgt[key].keys() and int(migsp) != 4:
            row = [ind_naics, ind_name, migsp, state_id_to_name[int(migsp)][0], missing_state_value, []]
            p1_all_state.append(row)
    
    # all states combined
    # migsp id 99
    asc_p1 = all_states_combinedp1[key]
    p1_all_state.append([ind_naics, ind_name, 99, "All States Combined", asc_p1, []])

In [48]:
np.unique(grouped_naicsp_state_wgt.keys().get_level_values(1))

array([ 1.,  2.,  5.,  6.,  8.,  9., 10., 11., 12., 13., 15., 16., 17.,
       18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.,
       31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 44.,
       45., 46., 47., 48., 49., 50., 51., 53., 54., 55., 56.])

In [49]:
p1_all_state

[['11',
  'Agriculture, Forestry, Fishing and Hunting',
  2.0,
  'Alaska',
  30.0,
  ['114']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  6.0,
  'California',
  249.0,
  ['111', '112', '113M']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  8.0,
  'Colorado',
  110.0,
  ['111', '112', '115']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  12.0,
  'Florida',
  15.0,
  ['111']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  16.0,
  'Idaho',
  22.0,
  ['113M']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  26.0,
  'Michigan',
  31.0,
  ['112']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  32.0,
  'Nevada',
  35.0,
  ['111']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  36.0,
  'New York',
  87.0,
  ['111', '115']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  37.0,
  'North Carolina',
  16.0,
  ['1133']],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  38.0,
  'North Dakota',

In [50]:
p1_all_state_df = pd.DataFrame(p1_all_state, columns=["NAICS", "Description", "MIGSP", "State", "MovedToAZLastYear", "NAICSPs"])
p1_all_state_df

,NAICS,Description,MIGSP,State,MovedToAZLastYear,NAICSPs
0,11,"Agriculture, Forestry, Fishing and Hunting",2.0,Alaska,30.0,[114]
1,11,"Agriculture, Forestry, Fishing and Hunting",6.0,California,249.0,"[111, 112, 113M]"
2,11,"Agriculture, Forestry, Fishing and Hunting",8.0,Colorado,110.0,"[111, 112, 115]"
3,11,"Agriculture, Forestry, Fishing and Hunting",12.0,Florida,15.0,[111]
4,11,"Agriculture, Forestry, Fishing and Hunting",16.0,Idaho,22.0,[113M]
...,...,...,...,...,...,...
1066,92,Government and Military,33.0,New Hampshire,NaN,[]
1067,92,Government and Military,44.0,Rhode Island,NaN,[]
1068,92,Government and Military,46.0,South Dakota,NaN,[]
1069,92,Government and Military,54.0,West Virginia,NaN,[]


In [51]:
# aggregate all industries over each state and get weight
extra_agg = p1_all_state_df.groupby("MIGSP")["MovedToAZLastYear"].sum()
extra_agg = list(map(lambda x: list(x), list(extra_agg.to_dict().items())))
for i,_ in enumerate(extra_agg):
    if extra_agg[i][0] != 4:
        extra_agg[i].insert(0, '0')
        extra_agg[i].insert(1, "All Industries Aggregated")
        extra_agg[i].insert(3, state_id_to_name[extra_agg[i][2]][0])
        extra_agg[i].insert(5,float('nan'))
extra_agg = pd.DataFrame(extra_agg, columns=["NAICS", "Description", "MIGSP", "State", "MovedToAZLastYear", "NAICSPs"])
p1_all_state_df = pd.concat([p1_all_state_df, extra_agg]).reset_index(drop=True)

In [52]:
p1_all_state_df.groupby('NAICS')["MovedToAZLastYear"].sum()

NAICS
0     235848.0
11      1664.0
21       914.0
22      1304.0
23     12424.0
31     15350.0
42      6366.0
44     25108.0
48      8882.0
51      4612.0
52     13830.0
53      4558.0
54     23142.0
55       448.0
56     14236.0
61     16682.0
62     30554.0
71      6354.0
72     14442.0
81      7888.0
90     10532.0
92     16558.0
Name: MovedToAZLastYear, dtype: float64

In [53]:
p1_all_state_df["PumsPopulation"] = float('nan')
for i in p1_all_state_df.index:
    naics = p1_all_state_df.iloc[i]["NAICS"]
    p1_all_state_df.loc[i, "PumsPopulation"] = total_weights_bynaics[naics]

In [54]:
p1_all_state_df.head()

,NAICS,Description,MIGSP,State,MovedToAZLastYear,NAICSPs,PumsPopulation
0,11,"Agriculture, Forestry, Fishing and Hunting",2.0,Alaska,30.0,[114],26587.0
1,11,"Agriculture, Forestry, Fishing and Hunting",6.0,California,249.0,"[111, 112, 113M]",26587.0
2,11,"Agriculture, Forestry, Fishing and Hunting",8.0,Colorado,110.0,"[111, 112, 115]",26587.0
3,11,"Agriculture, Forestry, Fishing and Hunting",12.0,Florida,15.0,[111],26587.0
4,11,"Agriculture, Forestry, Fishing and Hunting",16.0,Idaho,22.0,[113M],26587.0


In [55]:
p1_all_state_df = p1_all_state_df[["NAICS", "Description", "MIGSP", "State", "MovedToAZLastYear","PumsPopulation"]]

In [56]:
p1_all_state_df[p1_all_state_df.MovedToAZLastYear.isna()]

,NAICS,Description,MIGSP,State,MovedToAZLastYear,PumsPopulation
16,11,"Agriculture, Forestry, Fishing and Hunting",1.0,Alabama,NaN,26587.0
17,11,"Agriculture, Forestry, Fishing and Hunting",5.0,Arkansas,NaN,26587.0
18,11,"Agriculture, Forestry, Fishing and Hunting",9.0,Connecticut,NaN,26587.0
19,11,"Agriculture, Forestry, Fishing and Hunting",10.0,Delaware,NaN,26587.0
20,11,"Agriculture, Forestry, Fishing and Hunting",11.0,District of Columbia,NaN,26587.0
...,...,...,...,...,...,...
1065,92,Government and Military,19.0,Iowa,NaN,168397.0
1066,92,Government and Military,33.0,New Hampshire,NaN,168397.0
1067,92,Government and Military,44.0,Rhode Island,NaN,168397.0
1068,92,Government and Military,46.0,South Dakota,NaN,168397.0


In [57]:
total_weights_bynaics

ftNAICSP
11      26587.0
21      13419.0
22      30237.0
23     223614.0
31     220883.0
42      70814.0
44     337542.0
48     140118.0
51      53052.0
52     196657.0
53      76163.0
54     207076.0
55       5622.0
56     179950.0
61     273756.0
62     404457.0
71      67426.0
72     201013.0
81     143867.0
90     150293.0
92     168397.0
0     3190943.0
Name: PWGTP, dtype: float64

# p1 wide form

In [58]:
p1_all_state_df.MovedToAZLastYear.sum()

471696.0

In [59]:
# p1_all_state_df.NAICS = p1_all_state_df.NAICS.astype(str)

In [60]:
# new_cols = p1_all_state_df.NAICS.apply(lambda x: "MovedToAZLastYear_"+str(x)).unique()

In [61]:
# wide_df = pd.DataFrame(p1_all_state_df.State.unique(), columns=["State"])

In [62]:
# for col in new_cols:
#     wide_df[col] = ''
#     for i in wide_df.index:
#         st = wide_df.iloc[i]["State"]
#         wide_df.at[i,col] = p1_all_state_df[(p1_all_state_df.State==st) & (p1_all_state_df.NAICS==col.split('_')[1])].MovedToAZLastYear.values[0]
        

In [63]:
# wide_df.to_excel("pums_wideform_MovedToAZ_0810.xlsx")

# Part 2

In [64]:
grouped_by__socp_wgt = pums_outofaz.groupby(['SOCP'])["PWGTP"].sum()
grouped_by_socp_naicsp = pums_outofaz.groupby(['SOCP'])['NAICSP'].apply(lambda x : list(np.unique(list(x))))

In [65]:
# p2 = []
# for key in grouped_by_naics_socp_wgt.keys():
#     ind_name = industry_naics.loc[industry_naics.NAICS==key[0]]["Description"].values
#     if len(ind_name) == 0:
#         continue
#     else:
#         ind_name = ind_name[0]
#     ind_naics = key[0]
#     wgt = grouped_by_naics_socp_wgt[key]
#     naicsp = grouped_by_naics_socp_naicsp[key]
#     socp = key[1]
#     try:
#         socp_title = pums_titles.loc[pums_titles.SOCP==socp]["Title"].values[0]
#     except IndexError:
#         socp_title = 'Title Unavailable'
        
#     row = [ind_naics, ind_name, socp, socp_title, wgt, naicsp]
#     p2.append(row)


In [66]:
# p2_out = pd.DataFrame(p2, columns=["NAICS", "Description", "SOCP", "SOCP Title", "MovedToAZLastYear", "NAICSPs"])


In [67]:
# p2_out[p2_out["SOCP Title"]=='Title Unavailable']

In [68]:
# p2_out.to_csv("21_Part2_MovedToAZ_0705.csv", index=False)

### Part 2 all / Ignore broken

In [69]:
# p2_all = []
# for key in grouped_by_naics_wgt.keys():
#     ind_name = industry_naics.loc[industry_naics.NAICS==key[0]]["Description"].values
#     if len(ind_name) == 0:
#         ind_name = ''
#         ind_naics = ''
#     else:
#         ind_name = ind_name[0]
#         ind_naics = key[0]
#     wgt = grouped_by_naics_socp_wgt[key]
#     print(wgt)
#     naicsp = grouped_by_naics_socp_naicsp[key]
#     print(naicsp)
#     break
#     socp = key[1]
#     try:
#         socp_title = pums_titles.loc[pums_titles.SOCP==socp]["Title"].values[0]
#     except IndexError:
#         socp_title = 'Title Unavailable'
#     row = [ind_naics, ind_name, socp, socp_title, wgt, naicsp]
#     p2_all.append(row)


In [70]:
# p2_all_out = pd.DataFrame(p2_all, columns=["NAICS", "Description", "SOCP", "SOCP Title", "MovedToAZLastYear", "NAICSPs"])

In [71]:
# p2_all_out

In [72]:
# p2_all_out.to_csv("pums_Part2_MovedToAZ_0705.csv", index=False)

# Part 2 All by State

In [73]:
grouped_bysoc_state_wgt = pums_outofaz.groupby(['SOCP', 'MIGSP'])["PWGTP"].sum()
# grouped_bysoc_state_naicsp = pums_outofaz.groupby(['SOCP','MIGSP'])['NAICSP'].apply(lambda x : list(np.unique(list(x))))
all_states_combinedp2 = pums_outofaz.groupby(['SOCP'])['PWGTP'].sum()

In [74]:
grouped_bysoc_state_wgt

SOCP    MIGSP
111021  1.0       25.0
        6.0      240.0
        8.0      156.0
        11.0      14.0
        12.0     155.0
                 ...  
559830  37.0      51.0
        39.0      39.0
        42.0       2.0
        48.0     137.0
        51.0      60.0
Name: PWGTP, Length: 2775, dtype: float64

In [75]:
all_states_combinedp2

SOCP
111021    1117.0
1110XX    1140.0
112011      65.0
112021     871.0
112022     573.0
           ...  
5371XX      35.0
551010     449.0
552010     117.0
553010     418.0
559830     694.0
Name: PWGTP, Length: 432, dtype: float64

In [76]:
import tqdm

In [77]:
p2_all_state= []

for socp in tqdm.tqdm(grouped_bysoc_state_wgt.keys().get_level_values(0).unique()):
    #added 9.23 Total pums weight for occ
    total_wgt = total_weights_bysocp[socp]
    
    try:
        socp_title = pums_titles.loc[pums_titles.SOCP==socp]["Title"].values[0]
    except IndexError:
        socp_title = 'Title Unavailable'

    for migsp in grouped_bysoc_state_wgt[socp].keys():

        wgt = grouped_bysoc_state_wgt[socp, migsp]

        state_title = pums_migsp_titles.loc[pums_migsp_titles.MIGSP==migsp]["Name"].values[0]
        row = [socp, socp_title, migsp, state_title, wgt, total_wgt]
        p2_all_state.append(row)

    for i in pums_migsp_titles.index[:-1]:
        migsp = pums_migsp_titles.iloc[i]["MIGSP"]  
        if float(migsp) not in grouped_bysoc_state_wgt[socp].keys() and migsp != 4:
            row = [socp, socp_title,  migsp, 
                   pums_migsp_titles.loc[pums_migsp_titles.MIGSP==migsp]["Name"].values[0],
                   missing_state_value, total_wgt]
            p2_all_state.append(row)
    # All states combined
    acs_p3 = all_states_combinedp2[socp]
    p2_all_state.append([socp, socp_title, 99, "All States Combined", acs_p3, total_wgt])

100%|████████████████████████████████████████████████████████████████████████████████| 432/432 [00:15<00:00, 27.19it/s]


In [78]:
p2_all_state_df = pd.DataFrame(p2_all_state, columns=["SOCP", "SOCP Title", "MIGSP", "FromState", "MovedToAZLastYear", "PumsPopulation"])

In [79]:
p2_all_state_df[p2_all_state_df.MovedToAZLastYear.isna()]

,SOCP,SOCP Title,MIGSP,FromState,MovedToAZLastYear,PumsPopulation
21,111021,General And Operations Managers,2.0,Alaska,NaN,22977.0
22,111021,General And Operations Managers,5.0,Arkansas,NaN,22977.0
23,111021,General And Operations Managers,9.0,Connecticut,NaN,22977.0
24,111021,General And Operations Managers,10.0,Delaware,NaN,22977.0
25,111021,General And Operations Managers,15.0,Hawaii,NaN,22977.0
...,...,...,...,...,...,...
22026,559830,"Mil-Military, Rank Not Specified",50.0,Vermont,NaN,4936.0
22027,559830,"Mil-Military, Rank Not Specified",53.0,Washington,NaN,4936.0
22028,559830,"Mil-Military, Rank Not Specified",54.0,West Virginia,NaN,4936.0
22029,559830,"Mil-Military, Rank Not Specified",55.0,Wisconsin,NaN,4936.0


In [80]:
p2_all_state_df.MovedToAZLastYear.sum()

235848.0

In [81]:
temp_p2 = pd.DataFrame()
temp_grouped = p2_all_state_df.groupby(["MIGSP","FromState"]).agg({"MovedToAZLastYear":'sum'}).reset_index()

temp_p2["MIGSP"] = temp_grouped["MIGSP"]
temp_p2["FromState"] = temp_grouped["FromState"]
temp_p2["SOCP"] = '0'
temp_p2["SOCP Title"] = 'All Occupations Aggregated'
temp_p2["MovedToAZLastYear"] = temp_grouped["MovedToAZLastYear"]
temp_p2["PumsPopulation"] = total_weights_bysocp['0']
p2_all_state_final_df = pd.concat([p2_all_state_df, temp_p2])

In [82]:
p2_all_state_final_df

,SOCP,SOCP Title,MIGSP,FromState,MovedToAZLastYear,PumsPopulation
0,111021,General And Operations Managers,1.0,Alabama,25.0,22977.0
1,111021,General And Operations Managers,6.0,California,240.0,22977.0
2,111021,General And Operations Managers,8.0,Colorado,156.0,22977.0
3,111021,General And Operations Managers,11.0,District of Columbia,14.0,22977.0
4,111021,General And Operations Managers,12.0,Florida,155.0,22977.0
...,...,...,...,...,...,...
46,0,All Occupations Aggregated,53.0,Washington,6229.0,3190943.0
47,0,All Occupations Aggregated,54.0,West Virginia,367.0,3190943.0
48,0,All Occupations Aggregated,55.0,Wisconsin,1763.0,3190943.0
49,0,All Occupations Aggregated,56.0,Wyoming,827.0,3190943.0


In [83]:
p2_all_state_final_df.MovedToAZLastYear.sum()

471696.0

In [84]:
p2_all_state_final_df[p2_all_state_final_df["SOCP Title"] == 'Title Unavailable']

,SOCP,SOCP Title,MIGSP,FromState,MovedToAZLastYear,PumsPopulation


In [85]:
p2_all_state_final_df.to_excel(f"pums{pyear}_Part2_MovedToAZ_bystate_0901.xlsx", index=False)

## Part 3

In [86]:
us_state_codes = "AL AK AZ AR CA CO CT DE FL GA HI ID IL IN IA KS KY LA ME MD MA MI MN MS MO MT NE NV NH NJ NM NY NC ND OH OK OR PA RI SC SD TN TX UT VT VA WA WV WI WY"
us_state_codes = us_state_codes.split(" ")
us_state_codes.remove('AZ')

us_state_codes = list(map(lambda x: x.lower(), us_state_codes))

### Download PUMS States

In [87]:
import requests, zipfile, io
import os
# import time

In [88]:
#virinchi commented whole cell#

# dest = f"{pyear}/pums_states/"
# # os.makedirs(dest)
# if not os.path.exists(dest):
#     os.mkdir(dest)

# uri = mass_dl_url
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:104.0) Gecko/20100101 Firefox/104.0'}

# for sc in us_state_codes:
#     print(uri+sc+'.zip')
#     r = requests.get(uri+sc+'.zip', headers=headers)
#     print(sc, r.status_code)
#     z = zipfile.ZipFile(io.BytesIO(r.content))
#     z.extractall(f"{dest}{sc}")

In [89]:
from glob import glob

In [90]:
# moved_out_dict = {}

# for sc in us_state_codes:
#     file = glob(f"{dest}{sc}/ss16p*.csv")
#     df = pd.read_csv(file[0], usecols=["SOCP", "NAICSP", "PWGTP", "MIGSP"])
#     df = df[df.MIGSP==4]
#     df['NAICSP'] = df['NAICSP'].astype(str)
#     df['ftNAICSP'] = df["NAICSP"].apply(lambda x: x[:2])
    
#     df = df[df["ftNAICSP"].isin(["3M", "4M", "92"])==False]
#     df.loc[df["ftNAICSP"]=="32","ftNAICSP"] = "31" 
#     df.loc[df["ftNAICSP"]=="33","ftNAICSP"] = "31" 

#     df.loc[df["ftNAICSP"]=="45","ftNAICSP"] = "44"

#     df.loc[df["ftNAICSP"]=="49","ftNAICSP"] = "48"
    
#     df = df[df["ftNAICSP"]!="na"]
    
#     df.ftNAICSP = df.ftNAICSP.astype(str)
    
#     print(sc, df.shape)
    
#     naics_socp_agg = df.groupby(["ftNAICSP", "SOCP"])["PWGTP"].sum()
    
#     for key in naics_socp_agg.keys():
#         naics = key[0]
#         socp = key[1]
        
#         if naics not in moved_out_dict.keys():
#             moved_out_dict[naics] = {}
        
#         if socp not in moved_out_dict[naics].keys():
#             moved_out_dict[naics][socp] = 0
        
#         moved_out_dict[naics][socp] += naics_socp_agg[key]
    
# moved_out_dict

In [91]:
# # part 3 for only 21 industries
# aggd_21_ind_rows = []

# for i in industry_naics.index:
#     r = industry_naics.iloc[i]
#     if r.NAICS in moved_out_dict.keys():
#         for socp in moved_out_dict[r.NAICS]:
#             aggd_21_ind_rows.append([r.NAICS, r.Description, 
#                                      socp, 
#                                      pums_titles.loc[pums_titles.SOCP==socp]["Title"].values[0],
#                                      moved_out_dict[r.NAICS][socp]])
#     else:
#         aggd_21_ind_rows.append([r.NAICS, r.Description,'','',''])

# moved_out_21_df = pd.DataFrame(aggd_21_ind_rows, columns=["NAICS", "Description", "SOCP", "SOCP Title", "MovedOutOfAZLastYear"])

In [92]:
# moved_out_21_df.to_csv("21_Part3_MovedOutOfAZ_0805.csv", index=False)

# Part 3 By State

In [93]:
moved_out_dict = {}
file_prefix = 'psam_p'

for sc in us_state_codes:
    migsp = pums_migsp_titles[pums_migsp_titles.Short == sc.upper()]
#     print(f"{dest}{sc}/psam_p*.csv")
    file = glob(f"{dest}{sc}/{file_prefix}*.csv")
#     print(file)
    df = pd.read_csv(file[0], usecols=["AGEP", "ESR", "SOCP", "NAICSP", "PWGTP", "MIGSP"])
    
    #virinchi cell#
    df = df[ (df['AGEP']>20) & ((df['ESR'] == 1) | (df['ESR'] == 2) | (df['ESR'] == 4) | (df['ESR'] == 5) )]
    #virinchi cell#
    
    df = df[df.MIGSP==4]
    df['NAICSP'] = df['NAICSP'].astype(str)
    df['ftNAICSP'] = df["NAICSP"].apply(lambda x: x[:2])

    df.loc[df["ftNAICSP"]=="32","ftNAICSP"] = "31" 
    df.loc[df["ftNAICSP"]=="33","ftNAICSP"] = "31" 

    df.loc[df["ftNAICSP"]=="45","ftNAICSP"] = "44"

    df.loc[df["ftNAICSP"]=="49","ftNAICSP"] = "48"
    
    df.loc[df["ftNAICSP"]=="4M","ftNAICSP"] = "31" # manufacturing

    df.loc[df["ftNAICSP"]=="3M","ftNAICSP"] = "44"  # retail
    
    # govt codes
    temp_df = df.loc[df.ftNAICSP.isin(naics_titles[naics_titles.Title.str.startswith('Adm-')].NAICSP.apply(lambda x:x[:2]).values)]

    if temp_df.size > 0:
        temp_df.loc[:,"ftNAICSP"] = '90'
        df = pd.concat([df, temp_df])
        df = df.reset_index(drop=True)
    # # # # # #
    
    df = df[df["ftNAICSP"]!="na"]
    
    df.ftNAICSP = df.ftNAICSP.astype(str)
    
    print(sc, df.shape)
    
    naics_socp_agg = df.groupby(["ftNAICSP", "SOCP"])["PWGTP"].sum()
    sc = migsp.MIGSP.values[0]
    moved_out_dict[sc] = {}
    
    for key in naics_socp_agg.keys():
        naics = key[0]
        socp = key[1]
        
        if naics not in moved_out_dict[sc].keys():
            moved_out_dict[sc][naics] = {}
        
        if socp not in moved_out_dict[sc][naics].keys():
            moved_out_dict[sc][naics][socp] = 0
        
        moved_out_dict[sc][naics][socp] += naics_socp_agg[key]
    
moved_out_dict

C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\3585280033.py:9: DtypeWarning: Columns (97,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file[0], usecols=["AGEP", "ESR", "SOCP", "NAICSP", "PWGTP", "MIGSP"])
C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\3585280033.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.loc[:,"ftNAICSP"] = '90'


al (24, 7)


C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\3585280033.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.loc[:,"ftNAICSP"] = '90'


ak (36, 7)


C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\3585280033.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.loc[:,"ftNAICSP"] = '90'


...

wa (207, 7)
wv (3, 7)


C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\3585280033.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.loc[:,"ftNAICSP"] = '90'


wi (51, 7)
wy (23, 7)


C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\3585280033.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.loc[:,"ftNAICSP"] = '90'


{1: {'22': {'339030': 41},
  '23': {'472140': 45},
  '31': {'172011': 22, '439XXX': 22, '49904X': 11, '511011': 4},
  '42': {'5191XX': 5},
  '44': {'414010': 26, '537065': 6},
  '48': {'435052': 35, '533030': 11},
  '54': {'1191XX': 25, '151241': 47, '172051': 28},
  '56': {'259040': 13, '413091': 53},
  '61': {'251000': 8},
  '90': {'232011': 64, '532010': 7, '553010': 14},
  '92': {'232011': 64, '532010': 7, '553010': 14}},
 2: {'11': {'1940YY': 1},
  '23': {'472031': 36, '533030': 40},
  '42': {'5191XX': 17},
  '44': {'291051': 13, '37201X': 6},
  '48': {'397010': 3, '533054': 33},
  '56': {'37201X': 58, '397010': 41},
  '61': {'251000': 26, '259040': 30},
  '62': {'292061': 28, '319097': 41},
  '71': {'2530XX': 31, '272021': 18},
  '81': {'411012': 10, '4330XX': 23},
  '90': {'431011': 24,
   '499071': 15,
   '519160': 39,
   '537065': 15,
   '553010': 32,
   '559830': 38},
  '92': {'431011': 24,
   '499071': 15,
   '519160': 39,
   '537065': 15,
   '553010': 32,
   '559830': 38}},

In [94]:
vals = []
for k1 in moved_out_dict:
    for k2 in moved_out_dict[k1]:
        for k3 in moved_out_dict[k1][k2]:
            vals.append(moved_out_dict[k1][k2][k3])
np.sum(vals)

83978

In [95]:
p3_bystate = []
for migsp, vals_1 in moved_out_dict.items():
    state = pums_migsp_titles[pums_migsp_titles.MIGSP==migsp].Name.values[0]
    for naics, vals_2 in vals_1.items():
        naics_t = industry_naics[industry_naics.NAICS==naics].Description.values[0]
        for socp, pwgt in vals_2.items():
            try:
                soc_t = pums_titles[pums_titles.SOCP==socp].Title.values[0]
            except IndexError:
                soc_t = 'Title Unavailable'
            p3_bystate.append([naics, naics_t, socp, soc_t, migsp, state, pwgt])

In [96]:
p3bystate_df = pd.DataFrame(p3_bystate, columns=["NAICS", "Description", "SOCP", "SOCP Title", "MIGSP", "ToState", "LeftAZLastYear"])

In [97]:
p3bystate_df.LeftAZLastYear.sum()

83978

In [98]:
states_w_weights = p3bystate_df.groupby(["NAICS", "SOCP"])["MIGSP"].apply(list)

Append missing value to missing states

In [99]:
rows_p3_extra = []

for key in states_w_weights.keys():
    for migsp in pums_migsp_titles.MIGSP.values:
        if migsp not in states_w_weights[key] and migsp!='99':
            try:
                soc_t = pums_titles[pums_titles.SOCP==key[1]].Title.values[0]
            except IndexError:
                soc_t = 'Title Unavailable'
            rows_p3_extra.append([key[0], industry_naics[industry_naics.NAICS==key[0]].Description.values[0], 
                                 key[1],   soc_t    , migsp,
                                 pums_migsp_titles[pums_migsp_titles.MIGSP==migsp].Name.values[0], missing_state_value
                                 ])

In [100]:
rows_p3_extra

[['11',
  'Agriculture, Forestry, Fishing and Hunting',
  '119013',
  'Farmers, Ranchers, And Other Agricultural Managers',
  1,
  'Alabama',
  nan],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  '119013',
  'Farmers, Ranchers, And Other Agricultural Managers',
  2,
  'Alaska',
  nan],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  '119013',
  'Farmers, Ranchers, And Other Agricultural Managers',
  4,
  'Arizona',
  nan],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  '119013',
  'Farmers, Ranchers, And Other Agricultural Managers',
  5,
  'Arkansas',
  nan],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  '119013',
  'Farmers, Ranchers, And Other Agricultural Managers',
  9,
  'Connecticut',
  nan],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  '119013',
  'Farmers, Ranchers, And Other Agricultural Managers',
  10,
  'Delaware',
  nan],
 ['11',
  'Agriculture, Forestry, Fishing and Hunting',
  '119013',
  'Farmers, Ranchers, And 

In [101]:
p3_final_df = pd.concat([p3bystate_df, pd.DataFrame(rows_p3_extra, columns=["NAICS", "Description", "SOCP", "SOCP Title", "MIGSP", "ToState", "LeftAZLastYear"])])

In [102]:
p3_final_df = p3_final_df.reset_index(drop=True)

In [103]:
temp_p3 = pd.DataFrame()
# temp_p3["NAICS"] = [0]*len(p3_final_df.MIGSP.unique())
# temp_p3["Description"] = ['All Industries Aggregated']*len(p3_final_df.MIGSP.unique())
temp_grouped = p3_final_df.groupby(["SOCP","MIGSP","ToState"]).agg({"SOCP Title":'first', "LeftAZLastYear":'sum'}).reset_index()
temp_p3["MIGSP"] = temp_grouped["MIGSP"]
temp_p3["ToState"] = temp_grouped["ToState"]
temp_p3["SOCP"] = temp_grouped["SOCP"]
temp_p3["SOCP Title"] = temp_grouped["SOCP Title"]
temp_p3["LeftAZLastYear"] = temp_grouped["LeftAZLastYear"]
temp_p3["NAICS"] = 0
temp_p3["Description"] = 'All Industries Aggregated'
p3_final_df = pd.concat([p3_final_df, temp_p3])

In [104]:
p3_final_df = p3_final_df.reset_index(drop=True)

In [105]:
all_states_combinedp3 = p3_final_df.groupby(["NAICS", "Description", "SOCP", "SOCP Title"])["LeftAZLastYear"].sum().to_frame()
all_states_combinedp3 = all_states_combinedp3.reset_index()
all_states_combinedp3["MIGSP"] = 99
all_states_combinedp3["ToState"] = 'All States Combined'

In [106]:
p3_final_df = pd.concat([p3_final_df, all_states_combinedp3])
p3_final_df = p3_final_df.reset_index(drop=True)

### Consolidating p3 to only have SOCP as primary key

In [107]:
p3_consolidated = p3_final_df[p3_final_df.NAICS!=0]

In [108]:
p3_consolidated = p3_consolidated.groupby(["SOCP", "SOCP Title", "MIGSP", "ToState"], dropna=False)["LeftAZLastYear"].sum()
p3_consolidated = p3_consolidated.to_frame().reset_index()

In [109]:
p3_consolidated

,SOCP,SOCP Title,MIGSP,ToState,LeftAZLastYear
0,111021,General And Operations Managers,1,Alabama,0.0
1,111021,General And Operations Managers,2,Alaska,0.0
2,111021,General And Operations Managers,4,Arizona,0.0
3,111021,General And Operations Managers,5,Arkansas,0.0
4,111021,General And Operations Managers,6,California,150.0
...,...,...,...,...,...
20795,559830,"Mil-Military, Rank Not Specified",53,Washington,0.0
20796,559830,"Mil-Military, Rank Not Specified",54,West Virginia,0.0
20797,559830,"Mil-Military, Rank Not Specified",55,Wisconsin,0.0
20798,559830,"Mil-Military, Rank Not Specified",56,Wyoming,0.0


In [110]:
p3_consolidated.SOCP.unique().size, p3_final_df.SOCP.unique().size

(400, 400)

In [111]:
all_soc_combined = p3_consolidated.groupby(['MIGSP', 'ToState'])['LeftAZLastYear'].sum().to_frame().reset_index()

In [112]:
all_soc_combined['SOCP'] = '0'
all_soc_combined['SOCP Title'] = 'All Occupations Aggregated'

In [113]:
p3_consolidated_final = pd.concat([p3_consolidated, all_soc_combined], ignore_index=True).reset_index(drop=True)
p3_consolidated_final

,SOCP,SOCP Title,MIGSP,ToState,LeftAZLastYear
0,111021,General And Operations Managers,1,Alabama,0.0
1,111021,General And Operations Managers,2,Alaska,0.0
2,111021,General And Operations Managers,4,Arizona,0.0
3,111021,General And Operations Managers,5,Arkansas,0.0
4,111021,General And Operations Managers,6,California,150.0
...,...,...,...,...,...
20847,0,All Occupations Aggregated,53,Washington,5306.0
20848,0,All Occupations Aggregated,54,West Virginia,59.0
20849,0,All Occupations Aggregated,55,Wisconsin,1168.0
20850,0,All Occupations Aggregated,56,Wyoming,846.0


In [114]:
p3_consolidated_final.sort_values(["SOCP", "MIGSP"]).to_excel(f"pums{pyear}_Part3_MovedFromAZ_bystate_0901.xlsx", index=False)

# p3 by industry & state : p4

In [115]:
p3_final_df

,NAICS,Description,SOCP,SOCP Title,MIGSP,ToState,LeftAZLastYear
0,22,Utilities,339030,Security Guards And Gambling Surveillance Offi...,1,Alabama,41.0
1,23,Construction,472140,Painters And Paperhangers,1,Alabama,45.0
2,31,Manufacturing,172011,Aerospace Engineers,1,Alabama,22.0
3,31,Manufacturing,439XXX,Other Office And Administrative Support Workers,1,Alabama,22.0
4,31,Manufacturing,49904X,Industrial And Refractory Machinery Mechanics,1,Alabama,11.0
...,...,...,...,...,...,...,...
82887,92,Government and Military,537065,Stockers And Order Fillers,99,All States Combined,15.0
82888,92,Government and Military,551010,Mil-Military Officer Special And Tactical Oper...,99,All States Combined,308.0
82889,92,Government and Military,552010,Mil-First-Line Enlisted Military Supervisors,99,All States Combined,226.0
82890,92,Government and Military,553010,Adm-Military Enlisted Tactical Operations And ...,99,All States Combined,684.0


In [116]:
p3_byindstate_df = p3_final_df.groupby(["NAICS", "MIGSP"],as_index=False).agg({"Description":'first', "ToState":'first', "LeftAZLastYear":'sum'})

p3_byindstate_df[["NAICS", "Description", "MIGSP", "ToState", "LeftAZLastYear"]].to_excel(f"pums{pyear}_Part4_MovedFromAZ_bystate_0901.xlsx", index=False)


In [117]:
p3_byindstate_df[p3_byindstate_df.MIGSP==99]

,NAICS,MIGSP,Description,ToState,LeftAZLastYear
51,0,99,All Industries Aggregated,All States Combined,83978.0
103,11,99,"Agriculture, Forestry, Fishing and Hunting",All States Combined,902.0
155,21,99,"Mining, Quarrying, and Oil and Gas Extraction",All States Combined,514.0
207,22,99,Utilities,All States Combined,430.0
259,23,99,Construction,All States Combined,4370.0
311,31,99,Manufacturing,All States Combined,5678.0
363,42,99,Wholesale Trade,All States Combined,1655.0
415,44,99,Retail Trade,All States Combined,8758.0
467,48,99,Transportation and Warehousing,All States Combined,3116.0
519,51,99,Information,All States Combined,1543.0


In [118]:
p3_byindstate_df[p3_byindstate_df.NAICS==55].head()

,NAICS,MIGSP,Description,ToState,LeftAZLastYear


In [119]:
p3_byindstate_df.LeftAZLastYear.sum(), p3_final_df.LeftAZLastYear.sum()

(335912.0, 335912.0)

# Making final dfs

By Ind

In [120]:
# comnine df 4 & 1
p3_byindstate_df = p3_byindstate_df[p3_byindstate_df.ToState!='Arizona'][['NAICS', 'Description', 'MIGSP', 'ToState', 'LeftAZLastYear']].reset_index()
p1_all_state_df = p1_all_state_df.reset_index(drop=True)

In [121]:
p3_byindstate_df.NAICS = p3_byindstate_df.NAICS.astype(int)
p1_all_state_df.NAICS = p1_all_state_df.NAICS.astype(int) 

In [122]:
p1_all_state_df.MIGSP = p1_all_state_df.MIGSP.astype(int)
p3_byindstate_df.MIGSP = p3_byindstate_df.MIGSP.astype(int)

In [123]:
p1_all_state_df = p1_all_state_df.rename({'FromState': 'State'}, axis=1)
p3_byindstate_df = p3_byindstate_df.rename({'ToState': 'State'}, axis=1)

In [124]:
df_ind = p3_byindstate_df[p3_byindstate_df.columns]

In [125]:
p1_all_state_df

,NAICS,Description,MIGSP,State,MovedToAZLastYear,PumsPopulation
0,11,"Agriculture, Forestry, Fishing and Hunting",2,Alaska,30.0,26587.0
1,11,"Agriculture, Forestry, Fishing and Hunting",6,California,249.0,26587.0
2,11,"Agriculture, Forestry, Fishing and Hunting",8,Colorado,110.0,26587.0
3,11,"Agriculture, Forestry, Fishing and Hunting",12,Florida,15.0,26587.0
4,11,"Agriculture, Forestry, Fishing and Hunting",16,Idaho,22.0,26587.0
...,...,...,...,...,...,...
1117,0,All Industries Aggregated,53,Washington,6229.0,3190943.0
1118,0,All Industries Aggregated,54,West Virginia,367.0,3190943.0
1119,0,All Industries Aggregated,55,Wisconsin,1763.0,3190943.0
1120,0,All Industries Aggregated,56,Wyoming,827.0,3190943.0


In [126]:
df_ind = df_ind.merge(p1_all_state_df[["MIGSP", 'State', "NAICS", 'Description', "MovedToAZLastYear","PumsPopulation"]], on=["MIGSP", 'State', "NAICS", "Description"], how='outer', validate='1:1')
# df_ind.merge(p1_all_state_df[["MIGSP", 'State', "NAICS", 'Description', "MovedToAZLastYear"]], on=["MIGSP", 'State', "NAICS", "Description"], how='outer')

df_ind = df_ind.reset_index(drop=True)

In [127]:
# for i in p1_all_state_df.index:
#     try:
#         p1mig = p1_all_state_df.iloc[i]["MIGSP"]
#         p1nai = p1_all_state_df.iloc[i]["NAICS"]
#         p1wgt = p1_all_state_df.iloc[i]["MovedToAZLastYear"]
#         if p1nai == 999:
#             print(True)
#         df_ind.at[(df_ind.NAICS == p1nai) & (df_ind.MIGSP == p1mig), 'MovedToAZLastYear'] = p1wgt
#     except Exception as e:
#         print(e)

In [128]:
df_ind

,index,NAICS,Description,MIGSP,State,LeftAZLastYear,MovedToAZLastYear,PumsPopulation
0,0,0,All Industries Aggregated,1,Alabama,572.0,616.0,3190943.0
1,1,0,All Industries Aggregated,2,Alaska,781.0,985.0,3190943.0
2,3,0,All Industries Aggregated,5,Arkansas,437.0,591.0,3190943.0
3,4,0,All Industries Aggregated,6,California,14024.0,28408.0,3190943.0
4,5,0,All Industries Aggregated,8,Colorado,4969.0,6045.0,3190943.0
...,...,...,...,...,...,...,...,...
1117,1139,92,Government and Military,53,Washington,376.0,266.0,168397.0
1118,1140,92,Government and Military,54,West Virginia,0.0,NaN,168397.0
1119,1141,92,Government and Military,55,Wisconsin,26.0,90.0,168397.0
1120,1142,92,Government and Military,56,Wyoming,273.0,67.0,168397.0


In [129]:
df_ind.MovedToAZLastYear.sum(), p1_all_state_df.MovedToAZLastYear.sum()

(471696.0, 471696.0)

In [130]:
df_ind.LeftAZLastYear.sum(), p3_byindstate_df.LeftAZLastYear.sum()

(335912.0, 335912.0)

In [131]:
df_ind.PumsPopulation.sum(), p1_all_state_df.PumsPopulation.sum()

(325476186.0, 325476186.0)

In [132]:
df_ind.to_excel(f"pums{pyear}_ByInd.xlsx", index=False)

In [133]:
df_ind[df_ind.Description.isna()]

,index,NAICS,Description,MIGSP,State,LeftAZLastYear,MovedToAZLastYear,PumsPopulation


In [134]:
TEMPPPPPP = df_ind['NAICS'].astype(str) + '_' +  df_ind['MIGSP'].astype(str)

In [135]:
TEMPPPPPP.value_counts().sort_values()

0_1      1
11_23    1
0_13     1
0_20     1
0_19     1
        ..
92_42    1
92_41    1
92_40    1
92_47    1
92_99    1
Length: 1122, dtype: int64

**By occ**

In [136]:
p3_consolidated_final_merge = p3_consolidated_final[p3_consolidated_final.MIGSP!=4]

In [137]:
p3_consolidated_final_merge.MIGSP = p3_consolidated_final_merge.MIGSP.astype(int)
p2_all_state_final_df.MIGSP = p2_all_state_final_df.MIGSP.astype(int)

C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\1782107576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p3_consolidated_final_merge.MIGSP = p3_consolidated_final_merge.MIGSP.astype(int)


In [138]:
p3_consolidated_final_merge.SOCP = p3_consolidated_final_merge.SOCP.astype(str).apply(lambda x: x.strip()).values
p2_all_state_final_df.SOCP = p2_all_state_final_df.SOCP.astype(str).apply(lambda x: x.strip()).values

p3_consolidated_final_merge["SOCP Title"] = p3_consolidated_final_merge["SOCP Title"].astype(str).apply(lambda x: x.strip()).values
p2_all_state_final_df["SOCP Title"] = p2_all_state_final_df["SOCP Title"].astype(str).apply(lambda x: x.strip()).values

C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\670625937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p3_consolidated_final_merge.SOCP = p3_consolidated_final_merge.SOCP.astype(str).apply(lambda x: x.strip()).values
C:\Users\sjunuthu\AppData\Local\Temp\ipykernel_14188\670625937.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p3_consolidated_final_merge["SOCP Title"] = p3_consolidated_final_merge["SOCP Title"].astype(str).apply(lambda x: x.strip()).values


In [139]:
p2_all_state_final_df = p2_all_state_final_df.rename({'FromState': 'State'}, axis=1)
p3_consolidated_final_merge = p3_consolidated_final_merge.rename({'ToState': 'State'}, axis=1)

In [140]:
p2_all_state_final_df = p2_all_state_final_df.reset_index(drop=True)
p3_consolidated_final_merge = p3_consolidated_final_merge.reset_index(drop=True)

In [141]:
p3_consolidated_final_merge[(p3_consolidated_final_merge.MIGSP==99)&(p3_consolidated_final_merge.SOCP=='119011')]

,SOCP,SOCP Title,MIGSP,State,LeftAZLastYear


In [142]:
p2_all_state_final_df[(p2_all_state_final_df.MIGSP==99)&(p2_all_state_final_df.SOCP=='119012')]

,SOCP,SOCP Title,MIGSP,State,MovedToAZLastYear,PumsPopulation


In [143]:
# p2_all_state_final_df.merge(p3_final_df[["NAICS", "Description","MIGSP", "State", "SOCP", "SOCP Title", "LeftAZLastYear"]], on=["NAICS", "Description","MIGSP", "State", "SOCP", "SOCP Title"], how='outer')

df_occ = p2_all_state_final_df.merge(p3_consolidated_final_merge[["MIGSP", "State", "SOCP", "SOCP Title", "LeftAZLastYear"]], on=["MIGSP", "State", "SOCP", "SOCP Title"], how='outer', validate='1:1')


#p2_all_state_final_df.merge(p3_final_df[["NAICS", "MIGSP", "SOCP", "LeftAZLastYear"]], on=["NAICS", "MIGSP", "SOCP"], how='outer')


In [144]:
# make sure they are all 1
df_occ["KEYS"] = df_occ['SOCP'] + '_' + df_occ['MIGSP'].astype(str)

In [145]:
df_occ[df_occ["KEYS"]=='0_31']

,SOCP,SOCP Title,MIGSP,State,MovedToAZLastYear,PumsPopulation,LeftAZLastYear,KEYS
22058,0,All Occupations Aggregated,31,Nebraska,888.0,3190943.0,501.0,0_31


In [146]:
# validating row uniqueness
df_occ.KEYS.value_counts().sort_values()
# ALL VALUES MUST BE 1 - otherwise code is buggy and is duplicating some rows

111021_1     1
111021_13    1
111021_20    1
111021_25    1
111021_26    1
            ..
536051_56    1
536051_35    1
536051_34    1
536051_48    1
536051_99    1
Name: KEYS, Length: 24123, dtype: int64

In [147]:
df_occ.drop("KEYS", axis=1, inplace=True)

In [148]:
df_occ.LeftAZLastYear.sum(), p3_final_df.LeftAZLastYear.sum()

(335912.0, 335912.0)

In [149]:
df_occ.MovedToAZLastYear.sum(), p2_all_state_final_df.MovedToAZLastYear.sum()

(471696.0, 471696.0)

In [150]:
df_occ.PumsPopulation.sum(), p2_all_state_final_df.PumsPopulation.sum()

(321754461.0, 321754461.0)

In [151]:
df_occ

,SOCP,SOCP Title,MIGSP,State,MovedToAZLastYear,PumsPopulation,LeftAZLastYear
0,111021,General And Operations Managers,1,Alabama,25.0,22977.0,0.0
1,111021,General And Operations Managers,6,California,240.0,22977.0,150.0
2,111021,General And Operations Managers,8,Colorado,156.0,22977.0,19.0
3,111021,General And Operations Managers,11,District of Columbia,14.0,22977.0,0.0
4,111021,General And Operations Managers,12,Florida,155.0,22977.0,17.0
...,...,...,...,...,...,...,...
24118,536051,Transportation Inspectors,53,Washington,NaN,NaN,0.0
24119,536051,Transportation Inspectors,54,West Virginia,NaN,NaN,0.0
24120,536051,Transportation Inspectors,55,Wisconsin,NaN,NaN,0.0
24121,536051,Transportation Inspectors,56,Wyoming,NaN,NaN,0.0


In [152]:
# df_occ.at[df_occ.SOCP=='999920', "SOCP Title"] = "UNEMPLOYED AND LAST WORKED 5 YEARS AGO OR EARLIER OR NEVER WORKED".title()

In [153]:
df_occ.to_excel(f"pums{pyear}_ByOcc.xlsx", index=False)

# ⚠️⚠️⚠️ **<font color='red'>RUN ABOVE CELLS FOR ALL YEARS FIRST BEFORE PROCEEDING</font>** ⚠️⚠️⚠️

# Merging larger DFS

In [154]:
import  pandas as pd
import numpy as np

In [155]:
# years = [f"{x}5year" for x in [2020,2019,2018,2017,2016]]# [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
years = [f"{x}5year" for x in [2020]]# [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [156]:
years

['20205year']

In [157]:
# convert nans to 0 for net calc
def na2zero(cell):
    if np.isnan(cell):
        return 0
    else:
        return cell

**By Ind**

In [158]:
# By ind merge
dfs = []

for y in years:
    df = pd.read_excel(f"pums{y}_ByInd.xlsx")
    df.drop("index", axis=1, inplace=True)
    df.rename({"ToState": "State", 
               "LeftAZLastYear":f"LeftAZLastYear_{y}",
               "MovedToAZLastYear": f"MovedToAZLastYear_{y}",
               "PumsPopulation": f"PumsPopulation_{y}"}, axis=1, inplace=True)
    
    dfs.append(df)

In [159]:
merged = dfs[0]
for i in range(1,len(dfs)):
    merged = merged.merge(dfs[i][['NAICS', 'MIGSP', f'LeftAZLastYear_{years[i]}',
       f'MovedToAZLastYear_{years[i]}', f"PumsPopulation_{years[i]}"]], on=['NAICS', 'MIGSP'], how='outer')

In [160]:
dfs[0].columns

Index(['NAICS', 'Description', 'MIGSP', 'State', 'LeftAZLastYear_20205year',
       'MovedToAZLastYear_20205year', 'PumsPopulation_20205year'],
      dtype='object')

In [161]:
dfs[0].NAICS.unique().size

22

In [162]:
for i,y in enumerate(years):
    print(y, dfs[i][f'MovedToAZLastYear_{y}'].sum(), dfs[i][f'LeftAZLastYear_{y}'].sum(), dfs[i][f'PumsPopulation_{y}'].sum())

20205year 471696.0 335912 325476186


In [163]:
for i,y in enumerate(years):
    print(y, merged[f'MovedToAZLastYear_{y}'].sum(),merged[f'LeftAZLastYear_{y}'].sum(), merged[f'PumsPopulation_{y}'].sum())

20205year 471696.0 335912 325476186


In [164]:
# calucalting net migration and Percent migration
for i,y in enumerate(years):
    merged[f"NetMigration_{y}"] = merged[f"MovedToAZLastYear_{y}"].apply(na2zero) - merged[f'LeftAZLastYear_{y}'].apply(na2zero)
    merged[f"PercentMigration_{y}"] = merged[f"NetMigration_{y}"].apply(na2zero) / merged[f'PumsPopulation_{y}'].apply(na2zero)

In [165]:
merged

,NAICS,Description,MIGSP,State,LeftAZLastYear_20205year,MovedToAZLastYear_20205year,PumsPopulation_20205year,NetMigration_20205year,PercentMigration_20205year
0,0,All Industries Aggregated,1,Alabama,572,616.0,3190943,44.0,0.000014
1,0,All Industries Aggregated,2,Alaska,781,985.0,3190943,204.0,0.000064
2,0,All Industries Aggregated,5,Arkansas,437,591.0,3190943,154.0,0.000048
3,0,All Industries Aggregated,6,California,14024,28408.0,3190943,14384.0,0.004508
4,0,All Industries Aggregated,8,Colorado,4969,6045.0,3190943,1076.0,0.000337
...,...,...,...,...,...,...,...,...,...
1117,92,Government and Military,53,Washington,376,266.0,168397,-110.0,-0.000653
1118,92,Government and Military,54,West Virginia,0,NaN,168397,0.0,0.000000
1119,92,Government and Military,55,Wisconsin,26,90.0,168397,64.0,0.000380
1120,92,Government and Military,56,Wyoming,273,67.0,168397,-206.0,-0.001223


In [166]:
# year | movedin | movedout | netmoved

long_form_merge = pd.concat([merged[["NAICS", "Description", "MIGSP", "State"]]]*len(years), ignore_index=True)

In [167]:
long_form_merge

,NAICS,Description,MIGSP,State
0,0,All Industries Aggregated,1,Alabama
1,0,All Industries Aggregated,2,Alaska
2,0,All Industries Aggregated,5,Arkansas
3,0,All Industries Aggregated,6,California
4,0,All Industries Aggregated,8,Colorado
...,...,...,...,...
1117,92,Government and Military,53,Washington
1118,92,Government and Military,54,West Virginia
1119,92,Government and Military,55,Wisconsin
1120,92,Government and Military,56,Wyoming


In [168]:
year_col = np.concatenate([[y]*merged.shape[0] for y in years])
long_form_merge['Year'] = year_col
long_form_merge["MovedToAZLastYear"] = np.nan
long_form_merge["LeftAZLastYear"] = np.nan
long_form_merge["NetMigration"] = np.nan
long_form_merge["PumsPopulation"] = np.nan
long_form_merge["PercentMigration"] = np.nan

it = merged.shape[0]
for i in merged.index:
    for j, y in enumerate(years):
        naics = merged.iloc[i][f'NAICS']
        migsp = merged.iloc[i]['MIGSP']
        movedtoaz = merged.iloc[i][f'MovedToAZLastYear_{y}']
        leftaz = merged.iloc[i][f'LeftAZLastYear_{y}']
        netmig = merged.iloc[i][f'NetMigration_{y}']
        pumspop = merged.iloc[i][f'PumsPopulation_{y}']
        permig = merged.iloc[i][f'PercentMigration_{y}']
        
        
        
#         long_form_merge.at[(long_form_merge.NAICS == naics) & (long_form_merge.MIGSP == migsp) & (long_form_merge.Year == y), 
#                             f'MovedToAZLastYear'] = movedtoaz
#         long_form_merge.at[(long_form_merge.NAICS == naics) & (long_form_merge.MIGSP == migsp) & (long_form_merge.Year == y), 
#                             f'LeftAZLastYear'] = leftaz
#         long_form_merge.at[(long_form_merge.NAICS == naics) & (long_form_merge.MIGSP == migsp) & (long_form_merge.Year == y), 
#                            f'NetMigration'] = netmig
        
        # year_id * wideform_length + record index
        idx = j * it + i
        print(idx, naics, migsp, y, movedtoaz, leftaz, netmig, pumspop, permig)
        long_form_merge.loc[idx, ['MovedToAZLastYear', 'LeftAZLastYear', 'NetMigration', "PumsPopulation", "PercentMigration"]] = [movedtoaz, leftaz, netmig, pumspop, permig]

        
# j is the current year index 
# it is 1173 ~ 2446
# i curr index in merged dataset

0 0 1 20205year 616.0 572 44.0 3190943 1.378902725620608e-05
1 0 2 20205year 985.0 781 204.0 3190943 6.393094455150092e-05
2 0 5 20205year 591.0 437 154.0 3190943 4.826159539672128e-05
3 0 6 20205year 28408.0 14024 14384.0 3190943 0.004507758364847006
4 0 8 20205year 6045.0 4969 1076.0 3190943 0.0003372043938108578
5 0 9 20205year 555.0 242 313.0 3190943 9.809012570892053e-05
6 0 10 20205year 194.0 7 187.0 3190943 5.8603365838875843e-05
7 0 11 20205year 76.0 0 76.0 3190943 2.381741071526505e-05
8 0 12 20205year 4901.0 3351 1550.0 3190943 0.00048574982379816873
9 0 13 20205year 1718.0 1935 -217.0 3190943 -6.800497533174363e-05
10 0 15 20205year 847.0 507 340.0 3190943 0.00010655157425250154
11 0 16 20205year 1323.0 1505 -182.0 3190943 -5.703643092339788e-05
12 0 17 20205year 5648.0 2179 3469.0 3190943 0.0010871394443586113
13 0 18 20205year 1818.0 917 901.0 3190943 0.00028236167176912905
14 0 19 20205year 1039.0 930 109.0 3190943 3.415918115741961e-05
15 0 20 20205year 986.0 979 7.0 319

199 22 53 20205year 23.0 30 -7.0 30237 -0.0002315044481926117
200 22 54 20205year 56.0 0 56.0 30237 0.0018520355855408937
201 22 55 20205year nan 0 0.0 30237 0.0
202 22 56 20205year nan 0 0.0 30237 0.0
203 22 99 20205year 652.0 430 222.0 30237 0.007341998214108542
204 23 1 20205year 6.0 45 -39.0 223614 -0.0001744076846709061
205 23 2 20205year 83.0 76 7.0 223614 3.130394340247033e-05
206 23 5 20205year 11.0 23 -12.0 223614 -5.366390297566342e-05
207 23 6 20205year 1676.0 621 1055.0 223614 0.0047179514699437425
208 23 8 20205year 285.0 633 -348.0 223614 -0.0015562531862942392
209 23 9 20205year 17.0 19 -2.0 223614 -8.943983829277237e-06
210 23 10 20205year nan 0 0.0 223614 0.0
211 23 11 20205year nan 0 0.0 223614 0.0
212 23 12 20205year 187.0 138 49.0 223614 0.00021912760381729229
213 23 13 20205year 54.0 0 54.0 223614 0.0002414875633904854
214 23 15 20205year 7.0 30 -23.0 223614 -0.00010285581403668823
215 23 16 20205year 107.0 122 -15.0 223614 -6.707987871957928e-05
216 23 17 20205yea

519 52 13 20205year 183.0 143 40.0 196657 0.00020339982812714522
520 52 15 20205year 36.0 0 36.0 196657 0.00018305984531443071
521 52 16 20205year 44.0 84 -40.0 196657 -0.00020339982812714522
522 52 17 20205year 419.0 103 316.0 196657 0.0016068586422044472
523 52 18 20205year 122.0 64 58.0 196657 0.0002949297507843606
524 52 19 20205year 132.0 113 19.0 196657 9.661491836039399e-05
525 52 20 20205year 24.0 0 24.0 196657 0.00012203989687628713
526 52 21 20205year 25.0 17 8.0 196657 4.0679965625429044e-05
527 52 22 20205year nan 66 -66.0 196657 -0.0003356097164097896
528 52 23 20205year nan 0 0.0 196657 0.0
529 52 24 20205year 65.0 95 -30.0 196657 -0.00015254987109535894
530 52 25 20205year 55.0 0 55.0 196657 0.0002796747636748247
531 52 26 20205year 282.0 13 269.0 196657 0.0013678638441550516
532 52 27 20205year 99.0 18 81.0 196657 0.0004118846519574691
533 52 28 20205year 19.0 46 -27.0 196657 -0.00013729488398582304
534 52 29 20205year 241.0 4 237.0 196657 0.0012051439816533355
535 52 3

891 71 29 20205year 76.0 20 56.0 67426 0.0008305401477174977
892 71 30 20205year nan 5 -5.0 67426 -7.415537033191944e-05
893 71 31 20205year nan 0 0.0 67426 0.0
894 71 32 20205year 142.0 93 49.0 67426 0.0007267226292528105
895 71 33 20205year 23.0 21 2.0 67426 2.9662148132767776e-05
896 71 34 20205year 26.0 0 26.0 67426 0.00038560792572598106
897 71 35 20205year 84.0 59 25.0 67426 0.0003707768516595972
898 71 36 20205year 22.0 54 -32.0 67426 -0.0004745943701242844
899 71 37 20205year 27.0 26 1.0 67426 1.4831074066383888e-05
900 71 38 20205year 60.0 0 60.0 67426 0.0008898644439830333
901 71 39 20205year 41.0 0 41.0 67426 0.0006080740367217394
902 71 40 20205year 17.0 24 -7.0 67426 -0.00010381751846468721
903 71 41 20205year 24.0 74 -50.0 67426 -0.0007415537033191944
904 71 42 20205year 44.0 0 44.0 67426 0.0006525672589208911
905 71 44 20205year nan 30 -30.0 67426 -0.00044493222199151663
906 71 45 20205year 31.0 53 -22.0 67426 -0.00032628362946044554
907 71 46 20205year 6.0 0 6.0 67426 8

In [169]:
# CHANGE XXXX5year to XXXX
# convert year to true integer
try:
    long_form_merge.Year = long_form_merge.Year.apply(lambda x: x.replace('5year',''))
except Exception:
    pass

In [170]:
long_form_merge.head()

,NAICS,Description,MIGSP,State,Year,MovedToAZLastYear,LeftAZLastYear,NetMigration,PumsPopulation,PercentMigration
0,0,All Industries Aggregated,1,Alabama,2020,616.0,572.0,44.0,3190943.0,0.000014
1,0,All Industries Aggregated,2,Alaska,2020,985.0,781.0,204.0,3190943.0,0.000064
2,0,All Industries Aggregated,5,Arkansas,2020,591.0,437.0,154.0,3190943.0,0.000048
3,0,All Industries Aggregated,6,California,2020,28408.0,14024.0,14384.0,3190943.0,0.004508
4,0,All Industries Aggregated,8,Colorado,2020,6045.0,4969.0,1076.0,3190943.0,0.000337


In [171]:
long_form_merge.to_csv('pumsmig_5year_longform_merged_byind.csv', index=False)

In [172]:
long_form_merge.groupby('Year')[["MovedToAZLastYear", "LeftAZLastYear", "NetMigration", "PumsPopulation", "PercentMigration"]].sum()

,MovedToAZLastYear,LeftAZLastYear,NetMigration,PumsPopulation,PercentMigration
Year,,,,,
2020,471696.0,335912.0,135784.0,325476186.0,0.448458


In [173]:
print([[merged[f'MovedToAZLastYear_{y}'].sum(), merged[f'LeftAZLastYear_{y}'].sum(), merged[f'NetMigration_{y}'].sum(), merged[f'PumsPopulation_{y}'].sum(), merged[f'PercentMigration_{y}'].sum()]
      for y in years])

[[471696.0, 335912, 135784.0, 325476186, 0.4484578773834357]]


In [174]:
# movedin_16 | Movedout_16 | netmoved | ...
# merged.to_excel("pumsmig_merged_byind.xlsx", index=False)

In [175]:
del dfs


**By Occ**

In [176]:
#virinchi cell#
!pip install modin[ray]
#virinchi cell ended#

In [177]:
#virinchi# import modin.pandas as mpd

In [178]:

# By ind merge
dfs = []

for y in years:
    df = pd.read_excel(f"pums{y}_ByOcc.xlsx")
#     df.drop("index", axis=1, inplace=True)
    df.rename({"ToState": "State", 
               "LeftAZLastYear":f"LeftAZLastYear_{y}",
               "MovedToAZLastYear": f"MovedToAZLastYear_{y}",
               "PumsPopulation": f"PumsPopulation_{y}"}, axis=1, inplace=True)
    df.SOCP = df.SOCP.apply(str)
    dfs.append(df)

In [179]:
dfs[0].columns

Index(['SOCP', 'SOCP Title', 'MIGSP', 'State', 'MovedToAZLastYear_20205year',
       'PumsPopulation_20205year', 'LeftAZLastYear_20205year'],
      dtype='object')

In [180]:
base = pd.concat([t[["SOCP", "SOCP Title", "MIGSP", "State"]] for t in dfs], ignore_index=True).reset_index(drop=True).groupby(["SOCP", "MIGSP"]).agg({"SOCP Title": 'first', "State": 'first'}).reset_index()



In [181]:
base.shape

(24123, 4)

In [182]:
migsp_state = dfs[0].groupby('MIGSP')["State"].first()
len(migsp_state) * 631

32181

In [183]:
base["SOCP Title"] = base["SOCP Title"].apply(lambda x: x.replace('*',''))

In [184]:
base

,SOCP,MIGSP,SOCP Title,State
0,0,1,All Occupations Aggregated,Alabama
1,0,2,All Occupations Aggregated,Alaska
2,0,5,All Occupations Aggregated,Arkansas
3,0,6,All Occupations Aggregated,California
4,0,8,All Occupations Aggregated,Colorado
...,...,...,...,...
24118,559830,53,"Mil-Military, Rank Not Specified",Washington
24119,559830,54,"Mil-Military, Rank Not Specified",West Virginia
24120,559830,55,"Mil-Military, Rank Not Specified",Wisconsin
24121,559830,56,"Mil-Military, Rank Not Specified",Wyoming


In [185]:
merged = base
for i in range(0,len(dfs)):
    dfs[i] = dfs[i].drop(['SOCP Title', 'State'], axis=1)
    merged = merged.merge(dfs[i], on=['SOCP', 'MIGSP'], how='outer')

In [186]:
merged

,SOCP,MIGSP,SOCP Title,State,MovedToAZLastYear_20205year,PumsPopulation_20205year,LeftAZLastYear_20205year
0,0,1,All Occupations Aggregated,Alabama,616.0,3190943.0,572.0
1,0,2,All Occupations Aggregated,Alaska,985.0,3190943.0,781.0
2,0,5,All Occupations Aggregated,Arkansas,591.0,3190943.0,437.0
3,0,6,All Occupations Aggregated,California,28408.0,3190943.0,14024.0
4,0,8,All Occupations Aggregated,Colorado,6045.0,3190943.0,4969.0
...,...,...,...,...,...,...,...
24118,559830,53,"Mil-Military, Rank Not Specified",Washington,NaN,4936.0,0.0
24119,559830,54,"Mil-Military, Rank Not Specified",West Virginia,NaN,4936.0,0.0
24120,559830,55,"Mil-Military, Rank Not Specified",Wisconsin,NaN,4936.0,0.0
24121,559830,56,"Mil-Military, Rank Not Specified",Wyoming,NaN,4936.0,0.0


In [187]:
merged.shape

(24123, 7)

In [188]:
for i,y in enumerate(years):
    print(y, dfs[i][f'MovedToAZLastYear_{y}'].sum(), dfs[i][f'LeftAZLastYear_{y}'].sum())

20205year 471696.0 335912.0


In [189]:
for i,y in enumerate(years):
    print(y, merged[f'MovedToAZLastYear_{y}'].sum(),merged[f'LeftAZLastYear_{y}'].sum())

20205year 471696.0 335912.0


In [190]:
# calucalting net migration
for i,y in enumerate(years):
    merged[f"NetMigration_{y}"] = merged[f"MovedToAZLastYear_{y}"].apply(na2zero) - merged[f'LeftAZLastYear_{y}'].apply(na2zero)
    merged[f"PercentMigration_{y}"] = merged[f"NetMigration_{y}"].apply(na2zero) / merged[f'PumsPopulation_{y}'].apply(na2zero)

In [191]:
merged.shape

(24123, 9)

In [192]:
#irinchi# merged[merged["PercentMigration_20185year"].isna()]
merged[merged["PercentMigration_20205year"].isna()]

,SOCP,MIGSP,SOCP Title,State,MovedToAZLastYear_20205year,PumsPopulation_20205year,LeftAZLastYear_20205year,NetMigration_20205year,PercentMigration_20205year
2754,132081,1,"Tax Examiners And Collectors, And Revenue Agents",Alabama,NaN,NaN,0.0,0.0,NaN
2755,132081,2,"Tax Examiners And Collectors, And Revenue Agents",Alaska,NaN,NaN,0.0,0.0,NaN
2756,132081,5,"Tax Examiners And Collectors, And Revenue Agents",Arkansas,NaN,NaN,0.0,0.0,NaN
2758,132081,8,"Tax Examiners And Collectors, And Revenue Agents",Colorado,NaN,NaN,0.0,0.0,NaN
2759,132081,9,"Tax Examiners And Collectors, And Revenue Agents",Connecticut,NaN,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
23403,536051,51,Transportation Inspectors,Virginia,NaN,NaN,0.0,0.0,NaN
23404,536051,53,Transportation Inspectors,Washington,NaN,NaN,0.0,0.0,NaN
23405,536051,54,Transportation Inspectors,West Virginia,NaN,NaN,0.0,0.0,NaN
23406,536051,55,Transportation Inspectors,Wisconsin,NaN,NaN,0.0,0.0,NaN


In [193]:
# movedin_16 | Movedout_16 | netmoved | ...
# merged.to_excel("pumsmig_merged_byocc.xlsx", index=False)

In [194]:
# !export MODIN_ENGINE=dask

In [195]:
#virinchi# merged = mpd.DataFrame(merged)
merged = pd.DataFrame(merged)

In [196]:
#virinchi# long_form_merge = mpd.concat([merged[["MIGSP", "State", "SOCP", "SOCP Title"]]]*len(years), ignore_index=True)
long_form_merge = pd.concat([merged[["MIGSP", "State", "SOCP", "SOCP Title"]]]*len(years), ignore_index=True)
long_form_merge = long_form_merge.reset_index(drop=True)

In [197]:
toaz = [merged[f"MovedToAZLastYear_{y}"].values for y in years]
toaz = np.concatenate(toaz)
tostate = [merged[f"LeftAZLastYear_{y}"].values for y in years]
tostate = np.concatenate(tostate)
netmig = [merged[f"NetMigration_{y}"].values for y in years]
netmig = np.concatenate(netmig)
pumspop = [merged[f"PumsPopulation_{y}"].values for y in years]
pumspop = np.concatenate(pumspop)
permig = [merged[f"PercentMigration_{y}"].values for y in years]
permig = np.concatenate(permig)

In [198]:
year_col = np.concatenate([[y]*merged.shape[0] for y in years])
long_form_merge['Year'] = year_col

In [199]:
long_form_merge["MovedToAZLastYear"] = toaz
long_form_merge["LeftAZLastYear"] = tostate
long_form_merge["NetMigration"] = netmig
long_form_merge["PumsPopulation"] = pumspop
long_form_merge["PercentMigration"] = permig

In [200]:
long_form_merge.reset_index(drop=True)

,MIGSP,State,SOCP,SOCP Title,Year,MovedToAZLastYear,LeftAZLastYear,NetMigration,PumsPopulation,PercentMigration
0,1,Alabama,0,All Occupations Aggregated,20205year,616.0,572.0,44.0,3190943.0,0.000014
1,2,Alaska,0,All Occupations Aggregated,20205year,985.0,781.0,204.0,3190943.0,0.000064
2,5,Arkansas,0,All Occupations Aggregated,20205year,591.0,437.0,154.0,3190943.0,0.000048
3,6,California,0,All Occupations Aggregated,20205year,28408.0,14024.0,14384.0,3190943.0,0.004508
4,8,Colorado,0,All Occupations Aggregated,20205year,6045.0,4969.0,1076.0,3190943.0,0.000337
...,...,...,...,...,...,...,...,...,...,...
24118,53,Washington,559830,"Mil-Military, Rank Not Specified",20205year,NaN,0.0,0.0,4936.0,0.000000
24119,54,West Virginia,559830,"Mil-Military, Rank Not Specified",20205year,NaN,0.0,0.0,4936.0,0.000000
24120,55,Wisconsin,559830,"Mil-Military, Rank Not Specified",20205year,NaN,0.0,0.0,4936.0,0.000000
24121,56,Wyoming,559830,"Mil-Military, Rank Not Specified",20205year,NaN,0.0,0.0,4936.0,0.000000


In [201]:
long_form_merge.shape[0] / 15

1608.2

In [202]:
try:
    long_form_merge.Year = long_form_merge.Year.apply(lambda x: x.replace('5year',''))
except Exception:
    pass

In [203]:
long_form_merge[["SOCP", "SOCP Title", "MIGSP", "State", "Year", "MovedToAZLastYear", "LeftAZLastYear", "NetMigration", "PumsPopulation", "PercentMigration"]].to_csv('pumsmig_5year_longform_merged_byocc.csv', index=False)


In [204]:
# # year | movedin | movedout | netmoved

# long_form_merge = mpd.concat([merged[["NAICS", "Description", "MIGSP", "State", "SOCP", "SOCP Title"]]]*5, ignore_index=True)

# year_col = np.concatenate([[y]*merged.shape[0] for y in years])
# long_form_merge['Year'] = year_col
# long_form_merge["MovedToAZLastYear"] = np.nan
# long_form_merge["LeftAZLastYear"] = np.nan
# long_form_merge["NetMigration"] = np.nan

# for i in merged.index:
#     for y in years:
#         naics = merged.iloc[i][f'NAICS']
#         migsp = merged.iloc[i]['MIGSP']
#         socp = merged.iloc[i]['SOCP']
#         movedtoaz = merged.iloc[i][f'MovedToAZLastYear_{y}']
#         leftaz = merged.iloc[i][f'LeftAZLastYear_{y}']
#         netmig = merged.iloc[i][f'NetMigration_{y}']
        
#         long_form_merge.at[(long_form_merge.NAICS == naics) & (long_form_merge.MIGSP == migsp) & 
#                            (long_form_merge.SOCP == socp) & (long_form_merge.Year == y), 
#                             f'MovedToAZLastYear'] = movedtoaz
#         long_form_merge.at[(long_form_merge.NAICS == naics) & (long_form_merge.MIGSP == migsp) & 
#                            (long_form_merge.SOCP == socp) & (long_form_merge.Year == y), 
#                             f'LeftAZLastYear'] = leftaz
#         long_form_merge.at[(long_form_merge.NAICS == naics) & (long_form_merge.MIGSP == migsp) & 
#                            (long_form_merge.SOCP == socp) & (long_form_merge.Year == y), 
#                            f'NetMigration'] = netmig
# long_form_merge.to_csv('pumsmig_longform_merged_byocc.csv', index=False)

In [205]:
long_form_merge.head()

,MIGSP,State,SOCP,SOCP Title,Year,MovedToAZLastYear,LeftAZLastYear,NetMigration,PumsPopulation,PercentMigration
0,1,Alabama,0,All Occupations Aggregated,2020,616.0,572.0,44.0,3190943.0,0.000014
1,2,Alaska,0,All Occupations Aggregated,2020,985.0,781.0,204.0,3190943.0,0.000064
2,5,Arkansas,0,All Occupations Aggregated,2020,591.0,437.0,154.0,3190943.0,0.000048
3,6,California,0,All Occupations Aggregated,2020,28408.0,14024.0,14384.0,3190943.0,0.004508
4,8,Colorado,0,All Occupations Aggregated,2020,6045.0,4969.0,1076.0,3190943.0,0.000337


In [206]:
long_form_merge.groupby('Year')[["MovedToAZLastYear", "LeftAZLastYear", "NetMigration", "PumsPopulation", "PercentMigration"]].sum()


,MovedToAZLastYear,LeftAZLastYear,NetMigration,PumsPopulation,PercentMigration
Year,,,,,
2020,471696.0,335912.0,135784.0,321754461.0,NaN


In [207]:
print([[merged[f'MovedToAZLastYear_{y}'].sum(), merged[f'LeftAZLastYear_{y}'].sum(), merged[f'NetMigration_{y}'].sum(), merged[f'PumsPopulation_{y}'].sum(), merged[f'PercentMigration_{y}'].sum()]
      for y in years])

[[471696.0, 335912.0, 135784.0, 321754461.0, -inf]]
